<a href="https://colab.research.google.com/github/Yukselendincer/datasceinceproject/blob/main/Kredi_Skorlama_ve_Model_Validasyonu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Örnek Python kodu
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Veri simülasyonu
np.random.seed(42)
n_samples = 100000
data = {
    'age': np.random.randint(18, 70, n_samples),
    'income': np.random.lognormal(10, 0.5, n_samples),
    'debt_to_income': np.random.beta(2, 5, n_samples),
    'credit_score': np.random.randint(300, 850, n_samples),
    'employment_length': np.random.exponential(5, n_samples),
    'previous_defaults': np.random.poisson(0.3, n_samples),
    'loan_amount': np.random.uniform(1000, 100000, n_samples),
    'default_flag': np.random.binomial(1, 0.05, n_samples)  # %5 default oranı
}

In [ ]:
# Gerekli kütüphaneleri import ediyoruz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, classification_report
from scipy.stats import ks_2samp

# --- SANAL VERİ ÜRETİMİ (Bankacılık Simülasyonu) ---
np.random.seed(42)
n_samples = 2000

# Rastgele veri oluşturma
data = pd.DataFrame({
    'Yas': np.random.randint(18, 70, n_samples),
    'Gelir': np.random.normal(50000, 15000, n_samples),
    'Kredi_Tutari': np.random.normal(20000, 10000, n_samples),
    'Calisma_Suresi_Yil': np.random.randint(0, 30, n_samples),
    'Ev_Durumu': np.random.choice(['Kira', 'Ev_Sahibi', 'Aile_Yani'], n_samples)
})

# Hedef Değişken (Default - Batık) Oluşturma Mantığı
# Düşük gelir, genç yaş ve yüksek kredi borcu riski artırır (Simülasyon)
risk_score = (
    (data['Gelir'] * -0.0001) +
    (data['Yas'] * -0.5) +
    (data['Kredi_Tutari'] * 0.0002) +
    (data['Calisma_Suresi_Yil'] * -0.8) +
    np.random.normal(0, 10, n_samples) # Gürültü ekle
)

# En yüksek risk skoruna sahip %20'lik kesimi "Batık" (1) olarak etiketle
threshold = np.percentile(risk_score, 80)
data['Default'] = (risk_score > threshold).astype(int)

print("Veri Setinin İlk 5 Satırı:")
display(data.head())
print("\nSınıf Dağılımı:\n", data['Default'].value_counts())

In [ ]:
# WoE (Weight of Evidence) ve IV (Information Value) Hesaplama Fonksiyonu
def calculate_woe_iv(df, feature, target):
    lst = []
    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': df[df[feature] == val].count()[feature],
            'Good': df[(df[feature] == val) & (df[target] == 0)].count()[feature],
            'Bad': df[(df[feature] == val) & (df[target] == 1)].count()[feature]
        })

    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    # WoE Hesaplama (Sıfıra bölünme hatasını önlemek için küçük bir sayı eklenir)
    dset['WoE'] = np.log((dset['Distr_Good'] + 0.0001) / (dset['Distr_Bad'] + 0.0001))
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    dset = dset.sort_values(by='WoE')

    return dset

# Örnek olarak 'Yas' değişkenini gruplayıp WoE uygulayalım
# Yaşı 5 gruba ayırıyoruz (Binning)
data['Yas_Grup'] = pd.qcut(data['Yas'], q=5)
woe_df = calculate_woe_iv(data, 'Yas_Grup', 'Default')

print("Yaş Değişkeni için WoE ve IV Değerleri:")
display(woe_df[['Value', 'WoE', 'IV']])

# Görselleştirme
plt.figure(figsize=(10, 5))
sns.barplot(x='Value', y='WoE', data=woe_df, palette='viridis')
plt.title("Yaş Gruplarının Risk Ağırlığı (WoE)")
plt.xticks(rotation=45)
plt.show()

# Not: Modelde normalde ham veri yerine bu WoE değerleri kullanılır.
# Ancak basitlik adına model aşamasında ham veriyi kullanacağız.

In [ ]:
# Veri Hazırlığı
# Kategorik değişkenleri (Ev_Durumu) sayısal hale getir (One-Hot Encoding)
X = pd.get_dummies(data.drop(['Default', 'Yas_Grup'], axis=1), drop_first=True)
y = data['Default']

# Train - Test Ayrımı (%70 Eğitim, %30 Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Lojistik Regresyon Modeli Kurulumu
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Tahminler (Olasılık olarak)
y_pred_prob = model.predict_proba(X_test)[:, 1] # 1 olma (Batık) ihtimali

print("Model Katsayıları (Hangi değişken riski ne kadar etkiliyor?):")
coeffs = pd.DataFrame(zip(X.columns, model.coef_[0]), columns=['Feature', 'Coefficient'])
display(coeffs.sort_values(by='Coefficient'))

In [ ]:
# --- 1. ROC ve AUC (Gini Hesaplama) ---
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
auc_score = roc_auc_score(y_test, y_pred_prob)
gini_score = 2 * auc_score - 1  # Gini = 2*AUC - 1

print(f"AUC Skoru: {auc_score:.4f}")
print(f"Gini Katsayısı: {gini_score:.4f} (Bankacılıkta genelde 0.40+ iyi kabul edilir)")

# --- 2. KS (Kolmogorov-Smirnov) İstatistiği ---
# İyi ve Kötü müşterilerin olasılık dağılımlarını ayır
prob_good = y_pred_prob[y_test == 0]
prob_bad = y_pred_prob[y_test == 1]

ks_statistic, p_value = ks_2samp(prob_good, prob_bad)
print(f"KS İstatistiği: {ks_statistic:.4f} (Modelin ayırma gücü)")

# --- GÖRSELLEŞTİRME ---
plt.figure(figsize=(12, 5))

# ROC Eğrisi
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f'Model (Gini={gini_score:.2f})', color='blue')
plt.plot([0, 1], [0, 1], 'r--', label='Rastgele Seçim')
plt.title('ROC Eğrisi')
plt.xlabel('Yanlış Pozitif Oranı (FPR)')
plt.ylabel('Doğru Pozitif Oranı (TPR)')
plt.legend()

# KS Grafiği (Basitleştirilmiş Dağılım)
plt.subplot(1, 2, 2)
sns.kdeplot(prob_good, label='İyi Müşteriler (0)', shade=True, color='green')
sns.kdeplot(prob_bad, label='Kötü Müşteriler (1)', shade=True, color='red')
plt.title(f'KS Ayrımı (KS={ks_statistic:.2f})')
plt.xlabel('Tahmin Edilen Batma Olasılığı (PD)')
plt.legend()

plt.tight_layout()
plt.show()